<a href="https://colab.research.google.com/github/uto4ka101/First/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive"
data=pd.read_csv('/content/drive/MyDrive/train_new.csv',encoding='utf-8')

'Colab Notebooks'   recommender-systems-in-python-master
 data.csv	    RGBD_fruit_detection_faster-rcnn.pytorch-master
 Derevnja2021	    StudentsPerformance.csv
 Ng2022		    train_new.csv


In [ ]:
data.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Surname,Title,Sex,Age,FsizeD,Fsize,SibSp,Parch,Ticket,Fare,Deck,Embarked
0,0,1,0,3,Braund,Mr,male,22.0,small,2,1,0,A5,7.2500,X,S
1,1,2,1,1,Cumings,Mrs,female,38.0,small,2,1,0,PC,71.2833,C,C
2,2,3,1,3,Heikkinen,Miss,female,26.0,singleton,1,0,0,STONO2,7.9250,X,S
3,3,4,1,1,Futrelle,Mrs,female,35.0,small,2,1,0,X,53.1000,C,S
4,4,5,0,3,Allen,Mr,male,35.0,singleton,1,0,0,X,8.0500,X,S


In [ ]:
data.shape

(891, 16)

In [ ]:
data.isna() .sum()

Unnamed: 0       0
PassengerId      0
Survived         0
Pclass           0
Surname          0
Title            0
Sex              0
Age            177
FsizeD           0
Fsize            0
SibSp            0
Parch            0
Ticket           0
Fare             0
Deck             0
Embarked         2
dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imr = SimpleImputer(missing_values = np.nan, strategy='median')
imr = imr.fit(data[['Age']])
data['Age']=imr.transform(data[['Age']])

In [ ]:
imr = SimpleImputer(missing_values = np.nan, strategy ='most_frequent')
imr = imr.fit(data[['Embarked']])
data['Embarked']=imr.transform(data[['Embarked']])

In [ ]:
X = data.drop("Survived", axis = 1)
y = data.Survived

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 1.3 MB/s 


In [ ]:
from catboost import  CatBoostClassifier

In [ ]:
skf=KFold(n_splits=5)
print(skf.get_n_splits(X))
#cat_feature=[2,3,4,6,10,12,13]
cat_feature = np.where(X.dtypes != np.float)[0]
print(cat_feature)

5
[ 0  1  2  3  4  5  7  8  9 10 11 13 14]


In [ ]:
from catboost import CatBoostClassifier, Pool, cv
import catboost

In [ ]:
model = CatBoostClassifier(iterations = 100, depth = 3, learning_rate = 0.06, verbose = False)
f1 =[]
print(model)

In [ ]:
%%time
for train_index, test_index in skf.split(y):
   X_train = X.iloc[train_index]
   X_test = X.iloc [test_index]
   y_train = y[train_index]
   y_test = y[test_index]
#pool = Pool(data=X, label=y, cat_features=cat_features)
# test_pool = Pool(X_test, cat_features=cat_feature)
   train_pool = Pool(X_train,y_train, cat_features=cat_feature)
   test_pool = Pool(X_test, cat_features=cat_feature)
   model.fit(train_pool)
   preds=model.predict(test_pool, prediction_type='Class')
   f1.append(f1_score(y_test,preds.flatten()))

CPU times: user 950 ms, sys: 141 ms, total: 1.09 s
Wall time: 739 ms


In [ ]:
np.mean(f1)

0.7788358829482441

In [ ]:
score_feature = model.get_feature_importance(Pool(data=X_train, label=y_train, cat_features=cat_feature))


In [ ]:
data_score = pd.DataFrame({"feature_name": X.columns,"score": score_feature}) 
data_score = data_score.sort_values(by=["score"], ascending = False)

In [ ]:
data_score

,feature_name,score
4,Title,44.794293
2,Pclass,14.595184
5,Sex,14.333944
13,Deck,5.987194
3,Surname,5.914292
7,FsizeD,5.320903
12,Fare,2.576983
8,Fsize,2.033260
6,Age,1.722288
14,Embarked,1.217063


In [ ]:
new_columns = data_score.feature_name[data_score.score > 0.4]

In [ ]:
X_new_columns = X[new_columns]
X_new_columns 

,Title,Pclass,Sex,Deck,Surname,FsizeD,Fare,Fsize,Age,Embarked,SibSp,Parch
0,Mr,3,male,X,Braund,small,7.2500,2,22.0,S,1,0
1,Mrs,1,female,C,Cumings,small,71.2833,2,38.0,C,1,0
2,Miss,3,female,X,Heikkinen,singleton,7.9250,1,26.0,S,0,0
3,Mrs,1,female,C,Futrelle,small,53.1000,2,35.0,S,1,0
4,Mr,3,male,X,Allen,singleton,8.0500,1,35.0,S,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,Rare Title,2,male,X,Montvila,singleton,13.0000,1,27.0,S,0,0
887,Miss,1,female,B,Graham,singleton,30.0000,1,19.0,S,0,0
888,Miss,3,female,X,Johnston,small,23.4500,4,28.0,S,1,2
889,Mr,1,male,C,Behr,singleton,30.0000,1,26.0,C,0,0


In [ ]:
cat_feature=[X_new_columns.columns.get_loc(i) for i in X_new_columns.dtypes[X_new_columns.dtypes == "object"].index]

In [ ]:
f1_new = []

In [ ]:
%%time
for train_index, test_index in skf.split(y):
   X_train = X_new_columns.iloc[train_index]
   X_test = X_new_columns .iloc [test_index]
   y_train = y[train_index]
   y_test = y[test_index]
#pool = Pool(data=X, label=y, cat_features=cat_features)
# test_pool = Pool(X_test, cat_features=cat_feature)
   train_pool = Pool(data=X_train, label=y_train, cat_features=cat_feature)
   test_pool = Pool(data=X_test, label=y_test, cat_features=cat_feature)
   model.fit(train_pool, eval_set=test_pool, verbose=False)
   #model.fit(train_pool)
   preds =model.predict(test_pool)
   f1_new.append(f1_score(y_test,preds.flatten()))

CPU times: user 711 ms, sys: 96.5 ms, total: 808 ms
Wall time: 622 ms


In [ ]:
np.mean(f1_new)

0.7828776877718054